# 2026년 부동산 거래 가격 예측 (Prophet 활용)
- Prophet은 추세 + 계절성(반복 패턴) + 이상치 감지를 포함하여 시계열 데이터를 기반으로 2026년 값을 예측하는 데 적합함
- 거래량 top 30 주거지의 시계열 예측

## 1) 데이터 로드
- 데이터 : 정제된 서울시 부동산 실거래가 정보(2022~2025)

In [171]:
import pandas as pd
from prophet import Prophet

# 데이터 로드
file_path = "../Data/real-estate-prophet.csv"
df = pd.read_csv(file_path, encoding="utf-8")

# 데이터 확인
df.head()

,Unnamed: 0,접수연도,자치구명,법정동명,본번,부번,건물명,물건금액(만원),건물면적(㎡),층,...,신고한 개업공인중개사 시군구명,계약연도,계약월,계약일,위도,경도,지역,물건금액,거래일,지역+건물명+건물용도
0,0,2025,도봉구,방학동,638,10,한신빌라(638-10),14800,37.66,4,...,서울 도봉구,2025,1,17,37.667592,127.036982,도봉구 방학동,148000000,2025-01-17,도봉구 방학동 한신빌라(638-10) (연립다세대)
1,1,2025,강서구,등촌동,628,13,현대프린스텔,10300,26.16,6,...,서울 강서구,2025,1,17,37.560088,126.856361,강서구 등촌동,103000000,2025-01-17,강서구 등촌동 현대프린스텔 (오피스텔)
2,2,2025,관악구,신림동,1639,51,푸리마타운,12700,18.70,8,...,서울 관악구,2025,1,17,37.483391,126.927468,관악구 신림동,127000000,2025-01-17,관악구 신림동 푸리마타운 (오피스텔)
3,3,2025,구로구,고척동,339,0,고척파크푸르지오,84000,59.89,9,...,"서울 구로구, 서울 양천구",2025,1,17,37.507611,126.858113,구로구 고척동,840000000,2025-01-17,구로구 고척동 고척파크푸르지오 (아파트)
4,4,2025,광진구,중곡동,73,70,욱현하이브(73-70),19800,44.96,3,...,서울 동작구,2025,1,17,37.558748,127.095448,광진구 중곡동,198000000,2025-01-17,광진구 중곡동 욱현하이브(73-70) (연립다세대)


In [172]:
df.columns

Index(['Unnamed: 0', '접수연도', '자치구명', '법정동명', '본번', '부번', '건물명', '물건금액(만원)',
       '건물면적(㎡)', '층', '건축년도', '건물용도', '신고구분', '신고한 개업공인중개사 시군구명', '계약연도',
       '계약월', '계약일', '위도', '경도', '지역', '물건금액', '거래일', '지역+건물명+건물용도'],
      dtype='object')

## 2) 필요한 컬럼 선택

In [173]:
# 거래일 데이터
df['거래일'] = pd.to_datetime(df['거래일']).apply(lambda x: x.replace(day=1))
df['거래일']

0        2025-01-01
1        2025-01-01
2        2025-01-01
3        2025-01-01
4        2025-01-01
            ...    
223685   2020-04-01
223686   2018-07-01
223687   2016-06-01
223688   2016-04-01
223689   2013-01-01
Name: 거래일, Length: 223690, dtype: datetime64[ns]

In [174]:
# 필요한 컬럼 선택
df = df[['거래일', '지역+건물명+건물용도', '물건금액(만원)']]
df

,거래일,지역+건물명+건물용도,물건금액(만원)
0,2025-01-01,도봉구 방학동 한신빌라(638-10) (연립다세대),14800
1,2025-01-01,강서구 등촌동 현대프린스텔 (오피스텔),10300
2,2025-01-01,관악구 신림동 푸리마타운 (오피스텔),12700
3,2025-01-01,구로구 고척동 고척파크푸르지오 (아파트),84000
4,2025-01-01,광진구 중곡동 욱현하이브(73-70) (연립다세대),19800
...,...,...,...
223685,2020-04-01,강서구 내발산동 아이디움102동 (연립다세대),25400
223686,2018-07-01,서초구 서초동 진흥아파트 (아파트),155000
223687,2016-06-01,성북구 정릉동 진안빌라A(260-10) (연립다세대),35000
223688,2016-04-01,성북구 정릉동 진안빌라A(260-10) (연립다세대),35000


In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223690 entries, 0 to 223689
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   거래일          223690 non-null  datetime64[ns]
 1   지역+건물명+건물용도  223690 non-null  object        
 2   물건금액(만원)     223690 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 5.1+ MB


## 3) 매물이 가장 많은 상위 30개 건물 선정

In [176]:
# 매물이 가장 많은 상위 30개 아파트 선정
top_30_buildings = df['지역+건물명+건물용도'].value_counts().head(30).index
top_30_buildings

Index(['송파구 가락동 헬리오시티 (아파트)', '송파구 신천동 파크리오 (아파트)', '강동구 고덕동 고덕그라시움 (아파트)',
       '금천구 가산동 가산 시빅 (오피스텔)', '송파구 잠실동 리센츠 (아파트)', '강북구 미아동 에스케이북한산시티 (아파트)',
       '송파구 문정동 파크하비오 (오피스텔)', '강동구 상일동 고덕아르테온 (아파트)',
       '송파구 문정동 올림픽훼밀리타운 (아파트)', '송파구 잠실동 잠실엘스 (아파트)',
       '강서구 마곡동 힐스테이트에코 마곡나루역 라마다앙코르 서울마곡 (오피스텔)', '강동구 암사동 롯데캐슬퍼스트 (아파트)',
       '은평구 응암동 녹번역e편한세상캐슬 (아파트)', '강동구 고덕동 래미안힐스테이트고덕 (아파트)',
       '동대문구 답십리동 래미안위브 (아파트)', '강동구 둔촌동 올림픽파크 포레온 (아파트)',
       '은평구 대조동 베르디움STAY1 (아파트)', '성동구 하왕십리동 센트라스 (아파트)',
       '강동구 천호동 천호역 한강 푸르지오시티 (오피스텔)', '중구 신당동 남산타운 (아파트)',
       '송파구 잠실동 트리지움 (아파트)', '금천구 독산동 SHIN YOUNG (오피스텔)', '서초구 반포동 반포자이 (아파트)',
       '강남구 도곡동 도곡렉슬 (아파트)', '강서구 화곡동 강서힐스테이트 (아파트)',
       '송파구 가락동 가락(1차)쌍용아파트 (아파트)', '강서구 마곡동 마곡나루역보타닉푸르지오시티 (오피스텔)',
       '강남구 대치동 은마 (아파트)', '강동구 명일동 래미안솔베뉴 (아파트)', '도봉구 도봉동 한신 (아파트)'],
      dtype='object', name='지역+건물명+건물용도')

## 4) 각 건물별 시계열 예측 실행

In [177]:
df_building = df[df['지역+건물명+건물용도'] == '송파구 가락동 헬리오시티 (아파트)'].groupby('거래일').agg({'물건금액(만원)': 'mean'}).reset_index()
df_building = df_building.rename(columns={'거래일': 'ds', '물건금액(만원)': 'y'})  # Prophet 요구 형식 변환
df_building['y'] = df_building['y'].apply(lambda x: int(x))  # 소수점 제거
df_building

,ds,y
0,2021-12-01,127000
1,2022-01-01,237000
2,2022-02-01,158800
3,2022-03-01,211575
4,2022-04-01,165125
5,2022-05-01,187727
6,2022-06-01,202000
7,2022-07-01,196200
8,2022-08-01,190333
9,2022-09-01,166500


In [178]:
forecast_results = []

for building in top_30_buildings:
    df_building = df[df['지역+건물명+건물용도'] == building].groupby('거래일').agg({'물건금액(만원)':'mean'}).reset_index()  # 날짜별 평균 가격
    df_building = df_building.rename(columns={'거래일': 'ds', '물건금액(만원)': 'y'})  # Prophet 요구 형식
    df_building['y'] = df_building['y'].apply(lambda x: int(x))  # 소수점 제거

    # **데이터 개수 확인 후 2개 미만이면 스킵**
    if df_building.shape[0] < 2:
        print(f"Skipping {building}: Not enough data for prediction")
        continue  # 데이터가 부족하면 건너뜀

    # Prophet 모델 학습
    model = Prophet()
    model.fit(df_building)

    # 미래 데이터 생성 (2025년까지 예측)
    future = model.make_future_dataframe(periods=12, freq='M') # 12개월동안 예측, 월 단위로 미래 날짜 생성
    forecast = model.predict(future) # 예측결과

    # 2026년 데이터 필터링 및 저장
    forecast_2025 = forecast[forecast['ds'].dt.year == 2025][['ds', 'yhat']] # ds: 날짜, yhat: 예측값
    forecast_2025 = forecast_2025.rename(columns={'ds': '거래일', 'yhat': '물건금액(만원)'})  # 컬럼명 변경
    forecast_2025['지역+건물명+건물용도'] = building  # 건물 이름 추가
    forecast_results.append(forecast_2025)

16:08:43 - cmdstanpy - INFO - Chain [1] start processing
16:08:43 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:08:43 - cmdstanpy - INFO - Chain [1] start processing
16:08:44 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:08:44 - cmdstanpy - INFO - Chain [1] start processing
16:08:44 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:08:44 - cmdstanpy - INFO - C

Skipping 금천구 가산동 가산 시빅 (오피스텔): Not enough data for prediction


16:08:44 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:08:44 - cmdstanpy - INFO - Chain [1] start processing
16:08:44 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:08:44 - cmdstanpy - INFO - Chain [1] start processing
16:08:45 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:08:45 - cmdstanpy - INFO - Chain [1] start processing
16:08:45 - cmdstanpy - INFO - C

Skipping 은평구 대조동 베르디움STAY1 (아파트): Not enough data for prediction


16:09:00 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:09:00 - cmdstanpy - INFO - Chain [1] start processing
16:09:01 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:09:01 - cmdstanpy - INFO - Chain [1] start processing
16:09:01 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:09:01 - cmdstanpy - INFO - Chain [1] start processing
16:09:01 - cmdstanpy - INFO - C

Skipping 금천구 독산동 SHIN YOUNG (오피스텔): Not enough data for prediction


16:09:01 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:09:01 - cmdstanpy - INFO - Chain [1] start processing
16:09:01 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:09:01 - cmdstanpy - INFO - Chain [1] start processing
16:09:01 - cmdstanpy - INFO - Chain [1] done processing
/Users/gwondahyeon/Documents/GitHub/Real-Estate/env/lib/python3.12/site-packages/prophet/forecaster.py:1854: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

16:09:01 - cmdstanpy - INFO - Chain [1] start processing
16:09:01 - cmdstanpy - INFO - C

## 5) 모든 건물 예측 데이터 합치기 & 파일 저장 및 확인

In [179]:
# 모든 건물 예측 데이터 합치기
df_forecast_2025 = pd.concat(forecast_results, ignore_index=True)
df_forecast_2025['물건금액(만원)'] = df_forecast_2025['물건금액(만원)'].astype(int)  # 소수점 제거
df_forecast_2025

,거래일,물건금액(만원),지역+건물명+건물용도
0,2025-01-01,242221,송파구 가락동 헬리오시티 (아파트)
1,2025-01-31,211154,송파구 가락동 헬리오시티 (아파트)
2,2025-02-28,216085,송파구 가락동 헬리오시티 (아파트)
3,2025-03-31,203749,송파구 가락동 헬리오시티 (아파트)
4,2025-04-30,206359,송파구 가락동 헬리오시티 (아파트)
...,...,...,...
330,2025-08-31,63859,도봉구 도봉동 한신 (아파트)
331,2025-09-30,64036,도봉구 도봉동 한신 (아파트)
332,2025-10-31,56915,도봉구 도봉동 한신 (아파트)
333,2025-11-30,66239,도봉구 도봉동 한신 (아파트)


In [180]:
# 예측된 데이터 저장 (CSV 파일로 저장)
df_forecast_2025.to_csv("../Data/forecast_2025.csv", index=False)

# 데이터 확인 (상위 10개 출력)
print(df_forecast_2025.head(10))

         거래일  물건금액(만원)          지역+건물명+건물용도
0 2025-01-01    242221  송파구 가락동 헬리오시티 (아파트)
1 2025-01-31    211154  송파구 가락동 헬리오시티 (아파트)
2 2025-02-28    216085  송파구 가락동 헬리오시티 (아파트)
3 2025-03-31    203749  송파구 가락동 헬리오시티 (아파트)
4 2025-04-30    206359  송파구 가락동 헬리오시티 (아파트)
5 2025-05-31    226730  송파구 가락동 헬리오시티 (아파트)
6 2025-06-30    238993  송파구 가락동 헬리오시티 (아파트)
7 2025-07-31    237576  송파구 가락동 헬리오시티 (아파트)
8 2025-08-31    202417  송파구 가락동 헬리오시티 (아파트)
9 2025-09-30    237257  송파구 가락동 헬리오시티 (아파트)


## 6) 예측 데이터 시각화
- 2025년 예측 데이터 추가 & 그래프로 시각화
- 거래량 상위 3개 지역 예측값 확인

In [181]:
# 기존 데이터 로드
# file_path = '../Data/real-estate-prophet.csv'
# data1 = pd.read_csv(file_path, encoding="utf-8")
# data1 = data1[['거래일', '지역+건물명+건물용도', '물건금액(만원)']]
df['거래일'] = pd.to_datetime(df['거래일']).dt.date
df.head()

,거래일,지역+건물명+건물용도,물건금액(만원)
0,2025-01-01,도봉구 방학동 한신빌라(638-10) (연립다세대),14800
1,2025-01-01,강서구 등촌동 현대프린스텔 (오피스텔),10300
2,2025-01-01,관악구 신림동 푸리마타운 (오피스텔),12700
3,2025-01-01,구로구 고척동 고척파크푸르지오 (아파트),84000
4,2025-01-01,광진구 중곡동 욱현하이브(73-70) (연립다세대),19800


In [182]:
# 예측 데이터 로드
file_path = '../Data/forecast_2025.csv'
data2 = pd.read_csv(file_path, encoding="utf-8")
data2 = data2[['거래일', '지역+건물명+건물용도', '물건금액(만원)']]
data2['거래일'] = pd.to_datetime(data2['거래일']).dt.date
data2.head()

,거래일,지역+건물명+건물용도,물건금액(만원)
0,2025-01-01,송파구 가락동 헬리오시티 (아파트),242221
1,2025-01-31,송파구 가락동 헬리오시티 (아파트),211154
2,2025-02-28,송파구 가락동 헬리오시티 (아파트),216085
3,2025-03-31,송파구 가락동 헬리오시티 (아파트),203749
4,2025-04-30,송파구 가락동 헬리오시티 (아파트),206359


In [183]:
join_df = pd.concat([df, data2], ignore_index=True, axis=0)
join_df
join_df

,거래일,지역+건물명+건물용도,물건금액(만원)
0,2025-01-01,도봉구 방학동 한신빌라(638-10) (연립다세대),14800
1,2025-01-01,강서구 등촌동 현대프린스텔 (오피스텔),10300
2,2025-01-01,관악구 신림동 푸리마타운 (오피스텔),12700
3,2025-01-01,구로구 고척동 고척파크푸르지오 (아파트),84000
4,2025-01-01,광진구 중곡동 욱현하이브(73-70) (연립다세대),19800
...,...,...,...
224020,2025-08-31,도봉구 도봉동 한신 (아파트),63859
224021,2025-09-30,도봉구 도봉동 한신 (아파트),64036
224022,2025-10-31,도봉구 도봉동 한신 (아파트),56915
224023,2025-11-30,도봉구 도봉동 한신 (아파트),66239


In [184]:
# 가장 거래량이 많은 3개 건물 찾기
top_buildings = join_df['지역+건물명+건물용도'].value_counts().head(3).index  # 거래량 가장 많은 건물
top_buildings

Index(['송파구 가락동 헬리오시티 (아파트)', '송파구 신천동 파크리오 (아파트)', '강동구 고덕동 고덕그라시움 (아파트)'], dtype='object', name='지역+건물명+건물용도')

In [185]:
# 실거래가 추이 선 그래프
import plotly.express as px

# 가장 거래량이 많은 5개 건물 시각화 (예측 데이터 추가)
for top_building in top_buildings:
    apt = join_df[join_df['지역+건물명+건물용도'] == top_building].sort_values(by='거래일')

    apt['물건금액'] = apt['물건금액(만원)']*10000

    fig = px.line(apt, x='거래일', y='물건금액', title=f'{top_building} 거래가 변화 추이 [2021년 ~ 2025년(예측)]',
              labels={'거래일': '거래일', '물건금액': '거래 금액'}, markers=True)

    fig.update_layout(width=1200, height=600)
    fig.show()